**Run with conda env: allele_specific**

### Preparing the data for TxBurst inference 

In [51]:
cd /staging/leuven/stg_00041/Adrian/TALON_JANISZEWSKI_XCR2/

/lustre1/project/stg_00041/Adrian/TALON_JANISZEWSKI_XCR2


In [10]:
import pandas as pd
import numpy as np
import sys

## Calculate RPKMs manually with numpy
#### Import raw allelic count table 



In [11]:
allelic_mtx = pd.read_csv("/lustre1/project/stg_00041/Irene/Talon_Janiszewski_XCR2/XO_transcriptional_burst/X_loss_allelic_final.csv").sort_values(["GENE"]).reset_index()

In [12]:
allelic_mtx.head()

,index,Unnamed: 0,GENE,plate1_C07 . bi_allelic . Ca,plate1_D02 . bi_allelic . Ca,plate1_E08 . bi_allelic . Ca,plate2_A02 . cast . Ca,plate2_A03 . cast . Ca,plate2_A04 . cast . Ca,plate2_A05 . cast . Ca,...,plate2_G03 . bi_allelic . X129,plate2_G04 . bi_allelic . X129,plate2_G05 . cast . X129,plate2_G06 . bi_allelic . X129,plate2_H01 . bi_allelic . X129,plate2_H02 . cast . X129,plate2_H03 . bi_allelic . X129,plate2_H04 . bi_allelic . X129,plate2_H05 . cast . X129,plate2_H06 . bi_allelic . X129
0,0,1,0610009B22Rik,1,6,2,16,6,3,2,...,1,7,2,14,2,0,7,2,6,0
1,1,2,0610010F05Rik,0,0,0,0,5,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,3,0610010K14Rik,0,0,0,0,0,6,0,...,0,0,0,0,0,0,0,0,16,0
3,3,4,0610030E20Rik,1,0,0,0,0,0,0,...,0,9,0,2,1,0,0,0,3,0
4,4,5,0610038B21Rik,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [13]:
allelic_anno = pd.read_csv("allele_specific/txburst/genes.sorted.bed", sep = "\t", names=["chr","start","end","GENE"])

In [14]:
allelic_anno.head()

,chr,start,end,GENE
0,1,3073253,3074322,4933401J01Rik
1,1,3205901,3671498,Xkr4
2,1,3999557,4409241,Rp1
3,1,4490931,4497354,Sox17
4,1,4773206,4785739,MTrpl15


In [15]:
allelic_anno_match = allelic_anno[allelic_anno["GENE"] \
                                  .isin(allelic_mtx["GENE"])] \
                                  .drop_duplicates(subset=["GENE"]) \
                                  .sort_values(["GENE"]) \
                                  .reset_index()

In [16]:
allelic_anno_match.head()

,index,chr,start,end,GENE
0,3663,11,51685386,51688874,0610009B22Rik
1,3436,11,23564961,23633639,0610010F05Rik
2,4048,11,70235206,70237914,0610010K14Rik
3,22509,6,72347317,72353148,0610030E20Rik
4,26641,8,77517056,77523898,0610038B21Rik


In [17]:
# Assert matching between expr matrix and gene anno and extract matching index


pd.testing.assert_series_equal(allelic_mtx['GENE'], allelic_anno_match['GENE'])
allelic_mtx = allelic_mtx.set_index("GENE")
allelic_anno_match = allelic_anno_match.set_index("GENE")

matched_index = pd.Index.intersection(allelic_mtx.index, allelic_anno_match.index)

In [18]:
# Calculate gene lengths

allelic_anno_match["gene_len"] = allelic_anno_match["end"] - allelic_anno_match["start"]

In [19]:
allelic_anno_match.head()

,index,chr,start,end,gene_len
GENE,,,,,
0610009B22Rik,3663,11,51685386,51688874,3488
0610010F05Rik,3436,11,23564961,23633639,68678
0610010K14Rik,4048,11,70235206,70237914,2708
0610030E20Rik,22509,6,72347317,72353148,5831
0610038B21Rik,26641,8,77517056,77523898,6842


In [20]:
# Clean matrix

allelic_mtx = allelic_mtx.drop(["index","Unnamed: 0"], axis = 1)

In [21]:
allelic_mtx.head()

,plate1_C07 . bi_allelic . Ca,plate1_D02 . bi_allelic . Ca,plate1_E08 . bi_allelic . Ca,plate2_A02 . cast . Ca,plate2_A03 . cast . Ca,plate2_A04 . cast . Ca,plate2_A05 . cast . Ca,plate2_A06 . bi_allelic . Ca,plate2_B01 . mus . Ca,plate2_B02 . bi_allelic . Ca,...,plate2_G03 . bi_allelic . X129,plate2_G04 . bi_allelic . X129,plate2_G05 . cast . X129,plate2_G06 . bi_allelic . X129,plate2_H01 . bi_allelic . X129,plate2_H02 . cast . X129,plate2_H03 . bi_allelic . X129,plate2_H04 . bi_allelic . X129,plate2_H05 . cast . X129,plate2_H06 . bi_allelic . X129
GENE,,,,,,,,,,,,,,,,,,,,,
0610009B22Rik,1,6,2,16,6,3,2,2,0,5,...,1,7,2,14,2,0,7,2,6,0
0610010F05Rik,0,0,0,0,5,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
0610010K14Rik,0,0,0,0,0,6,0,0,0,0,...,0,0,0,0,0,0,0,0,16,0
0610030E20Rik,1,0,0,0,0,0,0,0,0,0,...,0,9,0,2,1,0,0,0,3,0
0610038B21Rik,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2


In [22]:
# Split expr matrix by the allele
allelic_mtx_129 = allelic_mtx.filter(like=". X129")
allelic_mtx_129.columns = allelic_mtx_129.columns.str.replace("\\.X129","")

allelic_mtx_Cast = allelic_mtx.filter(like=". Ca")
allelic_mtx_Cast.columns = allelic_mtx_Cast.columns.str.replace("\\.Ca","")

In [23]:
counts = np.asarray(allelic_mtx.loc[matched_index], dtype=int)

counts_129 = np.asarray(allelic_mtx_129.loc[matched_index], dtype=int)
counts_Cast = np.asarray(allelic_mtx_Cast.loc[matched_index], dtype=int)

gene_lengths = np.asarray(allelic_anno_match.loc[matched_index]['gene_len'],
                          dtype=int)

gene_names = np.array(matched_index)

In [24]:
print(counts.shape)
print(counts_129.shape)
print(counts_Cast.shape)
print(gene_lengths.shape)

(13947, 100)
(13947, 50)
(13947, 50)
(13947,)


In [25]:
# Function copied from Elegant NumPy book
# https://www.oreilly.com/library/view/elegant-scipy/9781491922927/ch01.html

def rpkm(counts, lengths):
    """Calculate reads per kilobase transcript per million reads.

    RPKM = (10^9 * C) / (N * L)

    Where:
    C = Number of reads mapped to a gene
    N = Total mapped reads in the experiment
    L = Exon length in base pairs for a gene

    Parameters
    ----------
    counts: array, shape (N_genes, N_samples)
        RNAseq (or similar) count data where columns are individual samples
        and rows are genes.
    lengths: array, shape (N_genes,)
        Gene lengths in base pairs in the same order
        as the rows in counts.

    Returns
    -------
    normed : array, shape (N_genes, N_samples)
        The RPKM normalized counts matrix.
    """
    N = np.sum(counts, axis=0)  # sum each column to get total reads per sample
    L = lengths
    C = counts

    normed = 1e9 * C / (N[np.newaxis, :] * L[:, np.newaxis])

    return(normed)

In [26]:
allelic_rpkm = rpkm(counts, gene_lengths)

allelic_129_rpkm = rpkm(counts_129, gene_lengths)
allelic_Cast_rpkm = rpkm(counts_Cast, gene_lengths)

In [27]:
allelic_rpkm_df = pd.DataFrame(data=allelic_rpkm, index=matched_index, columns=allelic_mtx.columns)

allelic_129_rpkm_df = pd.DataFrame(data=allelic_129_rpkm, index=matched_index, columns=allelic_mtx_129.columns)
allelic_Cast_rpkm_df = pd.DataFrame(data=allelic_Cast_rpkm, index=matched_index, columns=allelic_mtx_Cast.columns)

In [28]:
allelic_rpkm_df.to_csv("/lustre1/project/stg_00041/Irene/Talon_Janiszewski_XCR2/XO_transcriptional_burst/allelicXO_mtx_rpkm.csv")

allelic_129_rpkm_df.to_csv("/lustre1/project/stg_00041/Irene/Talon_Janiszewski_XCR2/XO_transcriptional_burst/allelicXO_mtx_129_rpkm.csv")
allelic_Cast_rpkm_df.to_csv("/lustre1/project/stg_00041/Irene/Talon_Janiszewski_XCR2/XO_transcriptional_burst/allelicXO_mtx_Cast_rpkm.csv")

### Distinguish missing data from no-expression

From Larsson TxBursts github:  
    “If you have estimated the allelic transcript counts from the fraction of allelic reads,  
    it is important to consider what is missing data in this case. Genes with expression (reads)  
    but no allelic reads are different from genes without expression (and therefore no allelic reads).  
    We handle the first case as missing data, since it is not possible to assign the expression.  
    In the second case, we replace the NaN with a 0 since there is genuinely no detected expression.  
    Omitting this step may have severe effects on the quality of the inference.”

In [29]:
# Import non allelic count matrix
counts_nonAllelic = pd.read_csv("/lustre1/project/stg_00041/Irene/Talon_Janiszewski_XCR2/XO_transcriptional_burst/X_loss_nonallelic_final.csv", 
                               index_col=0)

In [30]:
# Convert tables to a long format

allelic_129_rpkm_long = allelic_129_rpkm_df \
    .reset_index() \
    .melt(id_vars = "GENE", var_name = "cell", value_name="129_rpkm") 

allelic_Cast_rpkm_long = allelic_Cast_rpkm_df \
    .reset_index() \
    .melt(id_vars = "GENE", var_name = "cell", value_name="Cast_rpkm") 

counts_nonAllelic_long = counts_nonAllelic \
    .rename_axis("GENE") \
    .reset_index() \
    .melt(id_vars = "GENE", var_name = "cell", value_name = "nonAllelic") 

In [35]:
allelic_Cast_rpkm_long.head()

,GENE,cell,Cast_rpkm
0,0610009B22Rik,plate1_C07 . bi_allelic . Ca,3.318601
1,0610010F05Rik,plate1_C07 . bi_allelic . Ca,0.000000
2,0610010K14Rik,plate1_C07 . bi_allelic . Ca,0.000000
3,0610030E20Rik,plate1_C07 . bi_allelic . Ca,1.985128
4,0610038B21Rik,plate1_C07 . bi_allelic . Ca,0.000000


In [40]:
allelic_Cast_rpkm_long['cell'] = allelic_Cast_rpkm_long['cell'].str.replace(' . Ca', '')

In [41]:
allelic_Cast_rpkm_long.head()

,GENE,cell,Cast_rpkm
0,0610009B22Rik,plate1_C07 . bi_allelic,3.318601
1,0610010F05Rik,plate1_C07 . bi_allelic,0.000000
2,0610010K14Rik,plate1_C07 . bi_allelic,0.000000
3,0610030E20Rik,plate1_C07 . bi_allelic,1.985128
4,0610038B21Rik,plate1_C07 . bi_allelic,0.000000


In [36]:
allelic_129_rpkm_long.head()

,GENE,cell,129_rpkm
0,0610009B22Rik,plate1_C07 . bi_allelic . X129,8.393432
1,0610010F05Rik,plate1_C07 . bi_allelic . X129,0.000000
2,0610010K14Rik,plate1_C07 . bi_allelic . X129,0.000000
3,0610030E20Rik,plate1_C07 . bi_allelic . X129,21.756804
4,0610038B21Rik,plate1_C07 . bi_allelic . X129,0.000000


In [38]:
allelic_129_rpkm_long['cell'] = allelic_129_rpkm_long['cell'].str.replace(' . X129', '')

In [39]:
allelic_129_rpkm_long.head()

,GENE,cell,129_rpkm
0,0610009B22Rik,plate1_C07 . bi_allelic,8.393432
1,0610010F05Rik,plate1_C07 . bi_allelic,0.000000
2,0610010K14Rik,plate1_C07 . bi_allelic,0.000000
3,0610030E20Rik,plate1_C07 . bi_allelic,21.756804
4,0610038B21Rik,plate1_C07 . bi_allelic,0.000000


In [37]:
counts_nonAllelic_long.head()

,GENE,cell,nonAllelic
0,0610005C13Rik,plate1_C07 . bi_allelic,0
1,0610006L08Rik,plate1_C07 . bi_allelic,0
2,0610009B22Rik,plate1_C07 . bi_allelic,54
3,0610009E02Rik,plate1_C07 . bi_allelic,0
4,0610009L18Rik,plate1_C07 . bi_allelic,0


In [42]:
# Merge the allelic tables and calculate sumReads of the two alleles and allelic ratio (129/sumReads)

pd.testing.assert_series_equal(allelic_129_rpkm_long["GENE"],allelic_Cast_rpkm_long["GENE"])
pd.testing.assert_series_equal(allelic_129_rpkm_long["cell"],allelic_Cast_rpkm_long["cell"])

allelic_129_Cast_rpkm = allelic_129_rpkm_long \
                                .merge(
                                allelic_Cast_rpkm_long,
                                on=["GENE","cell"], how="left",
                                validate = "one_to_one") 

allelic_129_Cast_rpkm["sumReads"] = allelic_129_Cast_rpkm["129_rpkm"] + allelic_129_Cast_rpkm["Cast_rpkm"]

In [43]:
# Filter join with smaller nonAllelic table which is after seurat cell/gene filtering

counts_all = counts_nonAllelic_long \
                .merge(allelic_129_Cast_rpkm,
                      on=["GENE","cell"], how = "left")

In [44]:
# Handle missing data and no expression

## Genes with expression (reads, nonAllelic > 0) 
## but 
## no allelic reads (sumReads 0 or NaN) => NaN

# are different from 

## genes without expression (nonAllelic = 0) 
## and therefore 
## no allelic reads (sumReads = 0 or NaN) => 0


def data_handler(df, allele):
    
    """
    df is the name of the dataframe
    allele is the name of the column which will be corrected, either 129 or Cast
    """
    
    if df['nonAllelic'] > 0 and (df['sumReads'] == 0 or np.isnan(df['sumReads'])):
                               return float('NaN')
    if df['nonAllelic'] == 0 and (df['sumReads'] == 0 or np.isnan(df['sumReads'])):
                               return 0
    return df[allele]

In [45]:
# This is not time efficient and will take long
# For optimization vectorize with np.select

counts_all['129_rpkm_corr'] = counts_all.apply(data_handler, allele = '129_rpkm', axis=1)
counts_all['Cast_rpkm_corr'] = counts_all.apply(data_handler, allele = 'Cast_rpkm', axis=1)

In [46]:
counts_all.to_csv("/lustre1/project/stg_00041/Irene/Talon_Janiszewski_XCR2/XO_transcriptional_burst/allelicXO_counts_corrected.csv")

In [49]:
# Remove mitochondrial genes and GFP

counts_all_filt = counts_all[~counts_all["GENE"].str.contains('mt\\-|pCX-eGFP', na=False)]

# Remove Day 12 with Cast reactivated

#counts_all_filt = counts_all_filt[~counts_all_filt["cell"].str.contains('Day_12_Xi_Cast', na=False)]

In [50]:
counts_all_filt.head()

,GENE,cell,nonAllelic,129_rpkm,Cast_rpkm,sumReads,129_rpkm_corr,Cast_rpkm_corr
0,0610005C13Rik,plate1_C07 . bi_allelic,0,NaN,NaN,NaN,0.000000,0.000000
1,0610006L08Rik,plate1_C07 . bi_allelic,0,NaN,NaN,NaN,0.000000,0.000000
2,0610009B22Rik,plate1_C07 . bi_allelic,54,8.393432,3.318601,11.712033,8.393432,3.318601
3,0610009E02Rik,plate1_C07 . bi_allelic,0,NaN,NaN,NaN,0.000000,0.000000
4,0610009L18Rik,plate1_C07 . bi_allelic,0,NaN,NaN,NaN,0.000000,0.000000


In [56]:
counts_all_filt[['RNAid','Xstate']] = counts_all_filt.cell.str.split(" . ",expand=True,)

/data/leuven/331/vsc33185/yes/envs/Python_ML/lib/python3.6/site-packages/pandas/core/frame.py:3509: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [57]:
counts_all_filt.head()

,GENE,cell,nonAllelic,129_rpkm,Cast_rpkm,sumReads,129_rpkm_corr,Cast_rpkm_corr,RNAid,Xstate
0,0610005C13Rik,plate1_C07 . bi_allelic,0,NaN,NaN,NaN,0.000000,0.000000,plate1_C07,bi_allelic
1,0610006L08Rik,plate1_C07 . bi_allelic,0,NaN,NaN,NaN,0.000000,0.000000,plate1_C07,bi_allelic
2,0610009B22Rik,plate1_C07 . bi_allelic,54,8.393432,3.318601,11.712033,8.393432,3.318601,plate1_C07,bi_allelic
3,0610009E02Rik,plate1_C07 . bi_allelic,0,NaN,NaN,NaN,0.000000,0.000000,plate1_C07,bi_allelic
4,0610009L18Rik,plate1_C07 . bi_allelic,0,NaN,NaN,NaN,0.000000,0.000000,plate1_C07,bi_allelic


In [48]:
# Merge with information about clusters and pseuodotime

#cell_cluster_pseudo = pd.read_csv('allele_specific/txburst/seurat_data_pseudotime_clusters.csv')
#cell_cluster_pseudo = cell_cluster_pseudo[["Name","Pseudotime","State","seurat_clusters_rename"]].rename(columns={"Name":"cell", "seurat_clusters_rename":"cluster"})

#counts_all_filt = counts_all_filt \
                               # .merge(
                               # cell_cluster_pseudo,
                               # on=["cell"], how="left",
                               # validate = "many_to_one") 

In [58]:
counts_all_filt.to_csv("/lustre1/project/stg_00041/Irene/Talon_Janiszewski_XCR2/XO_transcriptional_burst/allelicXO_counts_final.csv")

In [59]:
def long_to_mtx(df_long,
                allele,
                Xstate=None):
    """ Convert long format gene expression back to genes x cells matrix for each 
    cell type or cluster for each allele. 
    
    Generates a valid input for txburstML.py
    
    timepoint: string
       A strings that will be used to grep timepoints from "cell" column
    cluster: string
        A string that selects the cluster from cluster column
    allele: string
       String that will indicate which allele to choose. Here, either: 'Cast_rpkm_corr'
           or '129_rpkm_corr'
    """
    if Xstate:   
        mtx = df_long[df_long['cell'].str.contains(Xstate)][['GENE','cell',allele]] \
            .pivot(index='GENE',columns='cell',values=allele)
    
        mtx.to_csv("/lustre1/project/stg_00041/Irene/Talon_Janiszewski_XCR2/XO_transcriptional_burst/{}_{}.csv".format(Xstate, allele))  
    #else:  
     #   mtx = df_long[df_long['cluster']==cluster][['GENE','cell',allele]] \
      #      .pivot(index='GENE',columns='cell',values=allele)
    
       # mtx.to_csv("allele_specific/txburst/clust_{}_{}.csv".format(cluster, allele))

In [60]:
Xstate = ['bi_allelic','cast','mus']
alleles = ['129_rpkm_corr', 'Cast_rpkm_corr']

for Xstate in Xstate:
    for allele in alleles:
        long_to_mtx(counts_all_filt, allele = allele, Xstate = Xstate)

In [363]:
#clusters = [0,1,2,3,4,5]
#alleles = ['129_rpkm_corr', 'Cast_rpkm_corr']

#for cluster in clusters:
 #   for allele in alleles:
  #      long_to_mtx(counts_all_filt, allele = allele, cluster = cluster)

In [291]:
# Prepare the table with selected sample and allele
# For starters Xa in Day 0 vs Xa in iPSCs

#D0_Xa = counts_all_filt[counts_all_filt['cell'].str.contains("plate")][['GENE','cell','Cast_rpkm_corr']] \
 #           .pivot(index='GENE',columns='cell',values='Cast_rpkm_corr')

#D12_Xa = counts_all_filt[counts_all_filt['cell'].str.contains("Day_12_Xi_Mus")][['GENE','cell','Cast_rpkm_corr']] \
#            .pivot(index='GENE',columns='cell',values='Cast_rpkm_corr')

#D0_Xa.to_csv("data/D0_Xa_rpkm.csv")
#D12_Xa.to_csv("data/D12_Xa_rpkm.csv")